<a href="https://colab.research.google.com/github/Yash-1812/ML_DL_Projects/blob/main/Movie_Suggestion_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('movies.csv', sep=',') # Assuming the delimiter is a comma

In [ ]:
df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [ ]:
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df.shape

(4803, 24)

In [ ]:
df.head(5)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
selected_features =['genres' , 'cast' , 'director' , 'keywords' , 'tagline']
selected_features

['genres', 'cast', 'director', 'keywords', 'tagline']

In [ ]:
for i in selected_features:
  df[i] = df[i].fillna('')

In [ ]:
combined_features = df['genres']+' '+df['cast']+' '+df['director']+' '+df['keywords']+' '+df['tagline']
combined_features  #TO CONVERT TEXT TO NUMERIC DATA WE NEED TO PUT THEM IN A SINGLE COLUMN

,0
0,Action Adventure Fantasy Science Fiction Sam W...
1,Adventure Fantasy Action Johnny Depp Orlando B...
2,Action Adventure Crime Daniel Craig Christoph ...
3,Action Crime Drama Thriller Christian Bale Mic...
4,Action Adventure Science Fiction Taylor Kitsch...
...,...
4798,Action Crime Thriller Carlos Gallardo Jaime de...
4799,Comedy Romance Edward Burns Kerry Bish\u00e9 M...
4800,Comedy Drama Romance TV Movie Eric Mabius Kris...
4801,Daniel Henney Eliza Coupe Bill Paxton Alan Ru...


In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510504
  (0, 274)	0.09021200873707366
  (0, 5274)	0.11108562744414444
  (0, 13599)	0.10364139873166359
  (0, 5437)	0.10364139873166359
  (0, 13349)	0.15021264094167083
  (0, 17007)	0.23643326319898794
  (0, 17290)	0.20197912553916564
  (0, 13319)	0.21774705394124838
  (0, 14064)	0.2059609041508414
  (0, 16668)	0.1984326396510037
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011813
  (0, 10229)	0.160586854000953
  (0, 13024)	0.19423620601088706
  (0, 7755)	0.11280357148547558
  (0, 2432)	0.17272411194153
  (0, 3678)	0.21392179219912874
  (0, 3065)	0.22208377802661422
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.3396275221095982
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997733
  (0, 14271)	0.21392179219912874
  (0, 4945)	0.24025852494110755
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.24713765026964


USING COSINE SIMILARITY TO CHECK THE SIMILARITY BETWEEN A MOVIE WITH ALL THE OTHER MOVIES IN THE LIST

In [ ]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
movie_name = input("Enter your favourite movie name : ")

Enter your favourite movie name : Ironman


In [ ]:
movie_name_list = df['title'].tolist()
movie_name_list

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

FINDING CLOSEST MATCH FOR THE MOVIE NAME PROVIDED BY THE USER

In [ ]:
find_close_match = difflib.get_close_matches(movie_name , movie_name_list)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [ ]:
index = df[df['title'] == close_match]['index'].values[0]
print(index)

68


In [ ]:
# Getting a set of similar movies
similarity_score = list(enumerate(similarity[index]))
similarity_score

[(0, np.float64(0.033570748780675445)),
 (1, np.float64(0.0546448279236134)),
 (2, np.float64(0.013735500604224321)),
 (3, np.float64(0.0064687561043920575)),
 (4, np.float64(0.03268943310073387)),
 (5, np.float64(0.013907256685755475)),
 (6, np.float64(0.07692837576335507)),
 (7, np.float64(0.23944423963486408)),
 (8, np.float64(0.007882387851851008)),
 (9, np.float64(0.07599206098164224)),
 (10, np.float64(0.07536074882460438)),
 (11, np.float64(0.01192606921174529)),
 (12, np.float64(0.013707618139948932)),
 (13, np.float64(0.01237607492508997)),
 (14, np.float64(0.09657127116284187)),
 (15, np.float64(0.007286271383816743)),
 (16, np.float64(0.22704403782296811)),
 (17, np.float64(0.013112928084103857)),
 (18, np.float64(0.04140526820609594)),
 (19, np.float64(0.07883282546834255)),
 (20, np.float64(0.07981173664799916)),
 (21, np.float64(0.011266873271064948)),
 (22, np.float64(0.006892575895462364)),
 (23, np.float64(0.006599097891242659)),
 (24, np.float64(0.012665208122549737))

In [ ]:
len(similarity_score)

4803

In [ ]:
# SORT THE SIMILARITY SCORE LIST FROM HIGHER SIMILARITY TO LOWER SIMILARITY VALUE
sorted_similarity_score = sorted(similarity_score , key = lambda x:x[1] , reverse = True)
sorted_similarity_score

[(68, np.float64(1.0000000000000002)),
 (79, np.float64(0.40890433998005965)),
 (31, np.float64(0.3146705244947752)),
 (7, np.float64(0.23944423963486408)),
 (16, np.float64(0.22704403782296811)),
 (26, np.float64(0.21566241096831162)),
 (85, np.float64(0.20615862984665334)),
 (182, np.float64(0.1957395613961161)),
 (511, np.float64(0.16702973947860683)),
 (3623, np.float64(0.1609246088135586)),
 (64, np.float64(0.1529992413944514)),
 (203, np.float64(0.1481866794866512)),
 (174, np.float64(0.1471993120942043)),
 (4401, np.float64(0.14505971470107848)),
 (101, np.float64(0.14401677581826292)),
 (46, np.float64(0.14216268867232232)),
 (169, np.float64(0.1380947013224906)),
 (1740, np.float64(0.1362438264169076)),
 (94, np.float64(0.13616819579029016)),
 (788, np.float64(0.13305895074229218)),
 (126, np.float64(0.13263982780511063)),
 (131, np.float64(0.13137698586006535)),
 (33, np.float64(0.13089810941050173)),
 (2487, np.float64(0.12309731939910509)),
 (783, np.float64(0.1216299556204

In [ ]:
# prompt: remove index , Name and dtype from the above code snippet

print("Suggested movies for you\n")
for i in range(10):
  movie = sorted_similarity_score[i][0]
  title = df[df['index'] == movie]['title'].values[0]
  print(title)

Suggested movies for you

Iron Man
Iron Man 2
Iron Man 3
Avengers: Age of Ultron
The Avengers
Captain America: Civil War
Captain America: The Winter Soldier
Ant-Man
X-Men
Made


In [ ]:
movie_name = input("Enter your favourite movie\n")

similarity = cosine_similarity(feature_vectors)
find_close_match = difflib.get_close_matches(movie_name, movie_name_list)
index = df[df['title'] == find_close_match[0]]['index'].values[0]
similarity_score = list(enumerate(similarity[index]))
sorted_similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
print("Suggested movies for you\n")
for i in range(10):
  movie = sorted_similarity_score[i][0]
  title = df[df['index'] == movie]['title'].values[0]
  print(title)

Enter your favourite movie
Spiderman
Suggested movies for you

Spider-Man
Spider-Man 3
Spider-Man 2
The Notebook
Seabiscuit
Clerks II
The Ice Storm
Oz: The Great and Powerful
Horrible Bosses
The Count of Monte Cristo
